# Assignment 3a: Basic Gradio RAG Frontend
## Day 6 Session 2 - Building Simple RAG Applications

In this assignment, you'll build a simple Gradio frontend for your RAG system with just the essential features:
- Button to initialize the vector database
- Search query input and button
- Display of AI responses

**Learning Objectives:**
- Create basic Gradio interfaces
- Connect RAG backend to frontend
- Handle user interactions and database initialization
- Build functional AI-powered web applications

**Prerequisites:**
- Completed Assignment 1 (Vector Database Basics)
- Completed Assignment 2 (Advanced RAG)
- Understanding of LlamaIndex fundamentals


## 📚 Part 1: Setup and Imports

Import all necessary libraries for building your Gradio RAG application.


In [ ]:
!pip install -r /requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.2 MB/s eta 0:00:00
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.2/39.2 MB 23.5 MB/s eta 0:

In [ ]:
# Import required libraries
import gradio as gr
from google.colab import userdata
import os
from pathlib import Path

# LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


## 🤖 Part 2: RAG Backend Class

Create a simple RAG backend that can initialize the database and answer queries.


In [ ]:
class SimpleRAGBackend:
    """Simple RAG backend for Gradio frontend."""

    def __init__(self):
        self.index = None
        self.setup_settings()

    def setup_settings(self):
        """Configure LlamaIndex settings."""
        # Set up the LLM using OpenRouter
        #api_key = os.getenv("OPENROUTER_API_KEY")
        api_key = userdata.get('OPENROUTER_API_KEY')
        if api_key:
            Settings.llm = OpenRouter(
                api_key=api_key,
                model="gpt-4o",
                temperature=0.1
            )
        else:
            # If API key is not set, explicitly set LLM to None for later check
            Settings.llm = None
            print("WARNING: OPENROUTER_API_KEY not found. LLM will not be initialized.")

        # Set up the embedding model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            trust_remote_code=True
        )

        # Set chunking parameters
        Settings.chunk_size = 512
        Settings.chunk_overlap = 50

    def initialize_database(self, data_folder="/content/data"):
        """Initialize the vector database with documents."""
        data_path = Path(data_folder)

        # Ensure data folder exists
        if not data_path.exists():
            data_path.mkdir(parents=True, exist_ok=True)
            print(f"Created data folder: {data_path}")

        # Create a dummy file if the folder is empty to ensure there's data to index
        if not any(data_path.iterdir()):
            dummy_file_path = data_path / "sample.txt"
            with open(dummy_file_path, "w") as f:
                f.write("This is a sample document for testing the RAG system. It contains information about general knowledge, for example, the capital of France is Paris. The largest ocean is the Pacific Ocean.")
            print(f"Created dummy file: {dummy_file_path} for testing.")

        try:
            # Check if LLM was properly set up (requires OPENROUTER_API_KEY)
            if Settings.llm is None:
                return "❌ OpenRouter API key not set. Please set the OPENROUTER_API_KEY environment variable to use the LLM."

            # Create vector store
            vector_store = LanceDBVectorStore(
                uri="/content/basic_rag_vectordb",
                table_name="documents"
            )

            # Load documents
            reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
            documents = reader.load_data()

            if not documents:
                return f"⚠️ No documents found in '{data_folder}' even after attempting to create a sample file. Please ensure there are text files in the directory."

            # Create storage context and index
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            self.index = VectorStoreIndex.from_documents(
                documents,
                storage_context=storage_context,
                show_progress=True
            )

            return f"✅ Database initialized successfully with {len(documents)} documents!"

        except Exception as e:
            return f"❌ Error initializing database: {str(e)}"

    def query(self, question):
        """Query the RAG system and return response."""
        # Check if index exists
        if self.index is None:
            return "❌ Please initialize the database first!"

        # Check if question is empty
        if not question or not question.strip():
            return "⚠️ Please enter a question first!"

        try:
            # Create query engine and get response
            query_engine = self.index.as_query_engine()
            response = query_engine.query(question)
            return str(response)

        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the backend
rag_backend = SimpleRAGBackend()
print("🚀 RAG Backend initialized and ready!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🚀 RAG Backend initialized and ready!


## 🎨 Part 3: Gradio Interface

Create a simple Gradio interface with:
1. Button to initialize the database
2. Text input for queries
3. Button to submit queries
4. Text output for responses
5. Text output for status messages


In [ ]:
def create_basic_rag_interface():
    """Create basic RAG interface with essential features."""

    def initialize_db():
        """Handle database initialization."""
        return rag_backend.initialize_database()

    def handle_query(question):
        """Handle user queries."""
        return rag_backend.query(question)

    # TODO: Create Gradio interface using gr.Blocks()
    # Hint: Look at the structure below and fill in the missing components

    with gr.Blocks(title="Basic RAG Assistant") as interface:
        # TODO: Add title and description
        # Hint: Use gr.Markdown() for formatted text
        gr.Markdown("# Basic RAG Assistant 🤖")
        gr.Markdown("This is a simple RAG (Retrieval Augmented Generation) system using LlamaIndex and Gradio.")

        # TODO: Add initialization section
        # Hint: You need to use gr.Button to initialize the database
        with gr.Row():
            init_btn = gr.Button("Initialize Database")

            # TODO: Add status output
            # Hint: You need to use gr.Textbox to display the status
            status_output = gr.Textbox(label="Status", interactive=False)

        # The connection between the button and the status output has already been implemented
        # at the end of this function

        #  status_output = ?

        # TODO: Add query section
        # Hint: You need a text input, submit button, and response output
        gr.Markdown("## Ask a Question")
        # Use gr.Textbox to create a text input
        query_input = gr.Textbox(label="Your Question", placeholder="e.g., What are the main topics in the documents?")

        # Use gr.Button to create a submit button
        submit_btn = gr.Button("Ask Question")

        # Use gr.Textbox to create a response output
        response_output = gr.Textbox(label="AI Response", interactive=False)

        # Connect buttons to functions
        # Uncomment when above is implemented
        init_btn.click(initialize_db, outputs=[status_output])
        submit_btn.click(handle_query, inputs=[query_input], outputs=[response_output])

    return interface

# Create the interface
basic_interface = create_basic_rag_interface()
print("✅ Basic RAG interface created successfully!")

✅ Basic RAG interface created successfully!


## 🚀 Part 4: Launch Your Application

Launch your Gradio application and test it!


In [ ]:
print("🎉 Launching your Basic RAG Assistant...")
print("🔗 Your application will open in a new browser tab!")
print("")
print("📋 Testing Instructions:")
print("1. Click 'Initialize Database' button first")
print("2. Wait for success message")
print("3. Enter a question in the query box")
print("4. Click 'Ask Question' to get AI response")
print("")
print("💡 Example questions to try:")
print("- What are the main topics in the documents?")
print("- Summarize the key findings")
print("- Explain the methodology used")
print("")
print("🚀 Launch your app:")

# Your launch code here:
basic_interface.launch()

🎉 Launching your Basic RAG Assistant...
🔗 Your application will open in a new browser tab!

📋 Testing Instructions:
1. Click 'Initialize Database' button first
2. Wait for success message
3. Enter a question in the query box
4. Click 'Ask Question' to get AI response

💡 Example questions to try:
- What are the main topics in the documents?
- Summarize the key findings
- Explain the methodology used

🚀 Launch your app:
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6eb6dc47438427e457.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## ✅ Assignment Completion Checklist

Before submitting, ensure you have:

- [x] RAG backend is provided and working
- [ ] Created Gradio interface with required components:
  - [ ] Title and description using gr.Markdown()
  - [ ] Initialize database button using gr.Button()
  - [ ] Status output using gr.Textbox()
  - [ ] Query input field using gr.Textbox()
  - [ ] Submit query button using gr.Button()
  - [ ] Response output area using gr.Textbox()
- [ ] Connected buttons to backend functions using .click()
- [ ] Successfully launched the application
- [ ] Tested the full workflow (initialize → query → response)

## 🎊 Congratulations!

You've successfully built your first Gradio RAG application! You now have:

- A functional web interface for your RAG system
- Understanding of Gradio basics and component connections
- A foundation for building more complex AI applications

**Next Steps**: Complete Assignment 3b to add advanced configuration options to your RAG interface!
